## Serializing Component

Autogen provides us a way via which we can easily serialize/deserialize components.

- **Serialization** : It is the process of converting an object into a stream of bytes or a string representation.
- **Deserialization** : It is the process of converting a stream of bytes or a string representation into an object.

In [10]:
import asyncio
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

## model client
model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')

#### This is useful for debudding, visualization and even when we want to share our work with others


## Termination Condition

Here we can convert the termination condition object into a json string which can be used for string and can be desirealized.

In [ ]:
max_message_termination = MaxMessageTermination(10)
text_mention_termination = TextMentionTermination("STOP")

final_termination = text_mention_termination & max_message_termination

## Serializing dumping the component to model
final_termination_config = final_termination.dump_component()

In [ ]:
## Serializing converting model to json string
final_termination_config.model_dump_json()

'{"provider":"autogen_agentchat.base.AndTerminationCondition","component_type":"termination","version":1,"component_version":1,"description":null,"label":"AndTerminationCondition","config":{"conditions":[{"provider":"autogen_agentchat.conditions.TextMentionTermination","component_type":"termination","version":1,"component_version":1,"description":"Terminate the conversation if a specific text is mentioned.","label":"TextMentionTermination","config":{"text":"STOP"}},{"provider":"autogen_agentchat.conditions.MaxMessageTermination","component_type":"termination","version":1,"component_version":1,"description":"Terminate the conversation after a maximum number of messages have been exchanged.","label":"MaxMessageTermination","config":{"max_messages":10,"include_agent_event":false}}]}}'

In [ ]:
## deserializing
new_final_termination = final_termination.load_component(final_termination_config)

In [8]:
new_final_termination

In [9]:
final_termination

## Agents

Now based on same idea we can serialize and deserialize agents as well.

In [13]:
agent = AssistantAgent(
    name = 'assistant',
    model_client=model_client,
    system_message='You are a helpful assistant who can solve to solve tasks.'
)

user_proxy = UserProxyAgent(name = 'user')

In [14]:
agent.dump_component()
user_proxy.dump_component()

ComponentModel(provider='autogen_agentchat.agents.UserProxyAgent', component_type='agent', version=1, component_version=1, description='An agent that can represent a human user through an input function.', label='UserProxyAgent', config={'name': 'user', 'description': 'A human user'})

#### Dumping the userproxy agent as a json file

In [15]:
import json

with open("user_proxy_dump.json","w") as file:
    json.dump(user_proxy.dump_component().model_dump(), file,indent=2)

#### Now loding the agent back from user_proxy_dump.json file.

In [16]:
with open('user_proxy_dump.json','r') as f:
    user_config = json.load(f)

In [17]:
user_agent_new = user_proxy.load_component(user_config)

## Team Serializing

In [18]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants=[agent, user_agent_new],
    termination_condition=new_final_termination
)

In [21]:
from pydantic import SecretStr
team_config = team.dump_component()

with open('team_dump.json','w') as f:
    json.dump(team_config.model_dump(), f, indent=2)